### Student Information

Name: 黃有璿

Student ID: 109062505

GitHub ID: anitahuang121

Kaggle name: Anita210503

Kaggle private scoreboard snapshot: 

<img src="pics/KAGGLE.jpg">

# Load data

In [ ]:
import pandas as pd

emotion_data = pd.read_csv("emotion.csv" , sep = "," , names = ["tweet_id" , "emotion"] , skiprows=[0])
identification_data = pd.read_csv("data_identification.csv" , sep = "," , names = ["tweet_id" , "identification"], skiprows=[0])
tweets_data = pd.read_json("tweets_DM.json" , lines = True)

In [ ]:
identification_data.loc[identification_data['identification'] == 'test']

In [ ]:
tweets_data.head()

In [ ]:
import json
from pandas.io.json import json_normalize

data_df = json_normalize(tweets_data._source)


In [ ]:
data_df = data_df.reindex(columns=['tweet.tweet_id','tweet.hashtags','tweet.text'])
data_df = data_df.rename(columns={'tweet.tweet_id':'tweet_id' , 'tweet.hashtags':'hashtags' , 'tweet.text':'text' })

In [ ]:
data_df

In [ ]:
data_merge_emotion = pd.merge(data_df, emotion_data , on=['tweet_id'], how="outer")

In [ ]:
data_all_info = pd.merge(data_merge_emotion, identification_data , on=['tweet_id'], how="outer")

In [ ]:
data_all_info

In [ ]:
[(data_all_info['identification'] == 'train') & (data_all_info['emotion'].isnull())]

In [ ]:
data_all_info.loc[data_all_info['identification'] == 'test']

In [ ]:
grouped = data_all_info.groupby(data_all_info.identification) 
train_df = grouped.get_group("train") 
test_df = grouped.get_group("test") 

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
## save to pickle file

train_df.to_pickle("train_df.pkl") 
test_df.to_pickle("test_df.pkl")


# Preprocessing

remove punctuation, noise such as '<LH>', numbers, stop_words
    
    and do stemming and lemmalization


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import CSVLogger
import nltk
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
import re
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

In [3]:
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
import re
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package wordnet to /home/hsnl-iot/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
import pandas as pd
import numpy as np

## load a pickle file
train_df = pd.read_pickle("train_df.pkl")
test_df = pd.read_pickle("test_df.pkl")


In [4]:
import string
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

train_df['text_without_punct'] = train_df['text'].apply(lambda x : remove_punctuation(x))
test_df['text_without_punct'] = test_df['text'].apply(lambda x : remove_punctuation(x))


In [5]:
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
tokenizer = RegexpTokenizer("[\w']+")
filt = '[!"#$%&()*,-./:;<=>?@[\]^_`{|}~ ]'

train_df['split'] = train_df['text'] .apply(lambda x: x.replace("<LH>",""))
train_df['split'] = train_df['split'].apply(lambda x: re.sub(filt, ' ', x.lower()))
train_df['split'] = train_df['split'].apply(lambda x: re.sub(r'\d+', " ", x))
train_df['split'] = train_df['split'].apply(lambda x: re.sub('<[^<]+?>', " ", x))
train_df['split'] = train_df['split'].apply(lambda x: x.split(" "))

test_df['split'] = test_df['text'] .apply(lambda x: x.replace("<LH>",""))
test_df['split'] = test_df['split'].apply(lambda x: re.sub(filt, ' ', x.lower()))
test_df['split'] = test_df['split'].apply(lambda x: re.sub(r'\d+', " ", x))
test_df['split'] = test_df['split'].apply(lambda x: re.sub('<[^<]+?>', " ", x))
test_df['split'] = test_df['split'].apply(lambda x: x.split(" "))


In [6]:
stop_words = set(stopwords.words('english'))
train_df['no_stopwords'] = train_df['split'].apply(lambda x: [item for item in x if item not in stop_words])
train_df['no_stopwords'] = train_df['no_stopwords'].apply(lambda x: [item for item in x if len(item) > 1])
test_df['no_stopwords'] = test_df['split'].apply(lambda x: [item for item in x if item not in stop_words])
test_df['no_stopwords'] = test_df['no_stopwords'].apply(lambda x: [item for item in x if len(item) > 1])

In [7]:
porter = nltk.stem.PorterStemmer()
lemmatizer = WordNetLemmatizer() 
train_df['stemmed'] = train_df['no_stopwords'].apply(lambda x: [porter.stem(y) for y in x])
test_df['stemmed'] = test_df['no_stopwords'].apply(lambda x: [porter.stem(y) for y in x])
train_df['lemmatization'] = train_df['no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
test_df['lemmatization'] = test_df['no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
train_df['lem_ste'] = train_df['lemmatization'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
test_df['lem_ste'] = test_df['lemmatization'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

In [8]:
train_df['clean'] = train_df['lem_ste'].apply(lambda x: [item for item in x if not item.isnumeric() ])
test_df['clean'] = test_df['lem_ste'].apply(lambda x: [item for item in x if not item.isnumeric() ])

In [9]:
train_df['cleaned'] = train_df['clean'].apply(' '.join)
test_df['cleaned'] = test_df['clean'].apply(' '.join)


In [6]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=15000, 
                             use_idf=True,  
                             tokenizer=tweet_tokenizer.tokenize,
                             stop_words=stopwords.words('english').append('lh')
                             )

tfidf_vectorizer.fit(train_df['cleaned'])

/home/hsnl-iot/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(max_features=15000,
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7ff2c6230850>>)

In [11]:
print(tfidf_vectorizer.get_feature_names()[:100])
print(tfidf_vectorizer.get_feature_names()[-100:])

["'", '+', '\\', 'a', "a's", 'aa', 'aaa', 'aajtak', 'aamir', 'aap', 'aaron', 'aaroncarter', 'aaronrodgers', 'aarti', 'ab', 'abandon', 'abandoned', 'abba', 'abbey', 'abby', 'abc', 'abcnetwork', 'abcnews', 'abcpolitics', 'aberdeen', 'abhlatino', 'abi', 'abide', 'abiding', 'ability', 'able', 'aboard', 'abominable', 'abomination', 'abortion', 'abound', 'about', 'aboveonly', 'abraham', 'abroad', 'absence', 'absent', 'absolute', 'absolutely', 'absorbed', 'absurd', 'abt', 'abu', 'abundance', 'abundant', 'abundantly', 'abuse', 'abused', 'abuser', 'abusing', 'abusive', 'abyss', 'ac', 'aca', 'academic', 'academy', 'acc', 'accent', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessible', 'accessory', 'accident', 'accidentally', 'accomplish', 'accomplished', 'accomplishing', 'accomplishment', 'accord', 'according', 'accordingly', 'account', 'accountability', 'accountable', 'accountant', 'accounting', 'acct', 'accuracy', 'accurate', 'accusation', 'accuse', '

In [12]:
#save cleaned data
train_df.to_pickle("train_cleaned_df.pkl") 
test_df.to_pickle("test_cleaned_df.pkl")

# Load cleaned data

In [5]:
train_df = pd.read_pickle("train_cleaned_df.pkl")
test_df = pd.read_pickle("test_cleaned_df.pkl")
train_df = train_df.sample(frac=1)

In [6]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=15000, 
                             use_idf=True,  
                             tokenizer=tweet_tokenizer.tokenize,
                             stop_words=stopwords.words('english').append('lh')
                             )

tfidf_vectorizer.fit(train_df['cleaned'])

/home/hsnl-iot/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(max_features=15000,
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7ff2c6230850>>)

# Final method that I used: Embedding + LSTM + Dense

## EMBEDDING

In [7]:
Dictionary = {}
for idx, token in enumerate(tfidf_vectorizer.get_feature_names()):
    Dictionary[token] = idx
    

In [8]:
from gensim.models import KeyedVectors
model_path = "GoogleNews/GoogleNews-vectors-negative300.bin.gz"
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [9]:
EMBEDDING_DIM = 300
def word2vec_matrix():
    matrix_emb = np.zeros((len(Dictionary)+1, EMBEDDING_DIM)) # replace MAX_WORD to len(Dictionary)
    for word, idx in Dictionary.items():
        try:
            vector =  word2vec_model.wv[word]
        except:
            vector = np.zeros(300,)
        if idx < 15000:
            matrix_emb[idx] = np.array(vector)
    matrix_emb[15000] = np.zeros(300,)     
    return matrix_emb

In [10]:
MAX_SEQUENCE_LENGTH = 50

def text_to_sequence(texts):
    tokenized_texts = []
    for string in texts:
        tokenized_text = []
        for idx, word in enumerate(string.split(' ')):
            
            # truncate
            if idx >= MAX_SEQUENCE_LENGTH: break 
                
            try:
                token = token_dict[word.lower()]
            except:
                token = 10000
            tokenized_text.append(token)
            
        # padding 
        if len(tokenized_text) < MAX_SEQUENCE_LENGTH: 
            tokenized_text.extend([10000]*(MAX_SEQUENCE_LENGTH-len(tokenized_text)))
                
        tokenized_texts.append(tokenized_text)

    return np.array(tokenized_texts)

In [11]:
from tensorflow.keras.utils import to_categorical
emotion_array = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise','trust']
def emotion_to_sequence(emotions):
    tokenized_emotions = []
    for string in emotions:
        token = emotion_array.index(string)
        tokenized_emotions.append(token)
    return to_categorical(tokenized_emotions)

In [12]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(emotion_array)

print('check label: ', label_encoder.classes_)
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)
def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']


In [13]:
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, GRU

In [14]:
train_df = train_df.sample(frac=1)

In [15]:
train_sequence = text_to_sequence(train_df['cleaned'])
train_label = emotion_to_sequence(train_df['emotion'])

test_sequence = text_to_sequence(test_df['cleaned'])


In [20]:
MAX_WORD = 15000
LSTM1_DIM = 256
DENSE1_DIM = 256
DENSE2_DIM = 256
CATEGORY_NUM = 8
BATCH_SIZE = 1000
VALIDATION_SPLIT = 0.2
EPOCHS = 6

csv_logger = CSVLogger('logs/training_log_LSTM.csv')
                    

def model_setting(texts, labels, matrix_emb="uniform"):    
    DROPOUT = 0.2
    
    if matrix_emb != "uniform": matrix_emb = Constant(matrix_emb)
    
#     split data to train & validation
    texts_train, texts_val, labels_train, labels_val = train_test_split(texts, labels, test_size=0.2, random_state=42)
    
    # model build
    model = Sequential()
    model.add(Embedding(MAX_WORD+1, EMBEDDING_DIM, embeddings_initializer=matrix_emb, input_length=MAX_SEQUENCE_LENGTH, trainable=True)) # with embedding matrix
    model.add(LSTM(LSTM1_DIM, dropout=DROPOUT, recurrent_dropout=DROPOUT))
    model.add(Dense(DENSE1_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dense(DENSE2_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dense(CATEGORY_NUM, kernel_initializer='normal', activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
    model.summary()

    
    history_callback = model.fit(
        texts_train, 
        labels_train, 
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE, 
        callbacks=[csv_logger],
        verbose=1, 
        shuffle=True, 
        validation_data=(texts_val, labels_val))
    
    pred_result = model.predict(test_sequence, batch_size=1000)
    
    pred_result_val = model.predict(texts_val, batch_size=1000)

    return history_callback, pred_result, labels_val, pred_result_val

In [ ]:
history, pred_result, labels_val, pred_result_val  = model_setting(train_sequence, train_label, word2vec_matrix())

<ipython-input-9-87177b11f884>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector =  word2vec_model.wv[word]
<ipython-input-20-1456ad931e06>:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if matrix_emb != "uniform": matrix_emb = Constant(matrix_emb)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           4500300   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 2056      
Total params: 5,204,308
Trainable params: 5,204,308
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
  19/1165 [..............................] - ETA: 11:14 - loss: 2.0772 - accuracy: 0.1770

In [ ]:
pred_result_val[:5]

In [ ]:
pred_result_val = label_decode(label_encoder, pred_result_val)
pred_result_val[:5]

In [ ]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, labels_val), pred_result_val), 2)))

In [ ]:
training_log = pd.DataFrame()
training_log = pd.read_csv("logs/training_log_LSTM.csv")
training_log

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train accuracy', 'Val accuracy'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train loss', 'Val loss'], loc='upper left')
plt.show()

In [ ]:
pred_form = pd.read_pickle("test_df.pkl")
pred_form = pred_form.drop('hashtags', axis=1)
pred_form = pred_form.drop('text', axis=1)
pred_form = pred_form.drop('identification', axis=1)
pred_form.rename(columns={'tweet_id':'id'}, inplace=True)

pred_form['emotion'] = label_decode(label_encoder, pred_result)
pred_form.to_csv('prediction_lstm.csv', index=False)

In [ ]:
from sklearn.metrics import classification_report
labels_val_decode = label_decode(label_encoder, labels_val)
pred_result_val_decode = label_decode(label_encoder, pred_result_val)
print(classification_report(y_true=labels_val_decode, y_pred=pred_result_val_decode))

# Other method I have tried

# METHOD1 : DECISION TREE
the result is not good, the best accuracy = 0.32959

In [ ]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
tfidf_vectorizer.fit(train_df['text'])

train_data_TFIDF_features_10000 = tfidf_vectorizer.transform(train_df['text'])

## check dimension
train_data_TFIDF_features_10000.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
X_train = tfidf_vectorizer.transform(train_df['text'])
y_train = train_df['emotion']

X_test = tfidf_vectorizer.transform(test_df['text'])
y_test = test_df['emotion']

## take a look at data dimension is a good habbit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)


In [ ]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:10]

In [ ]:
pred_form = pd.read_pickle("test_df.pkl")
pred_form = pred_form.drop('hashtags', axis=1)
pred_form = pred_form.drop('text', axis=1)
pred_form = pred_form.drop('identification', axis=1)
pred_form.rename(columns={'tweet_id':'id'}, inplace=True)

In [ ]:
pred_form['emotion'] = y_test_pred
pred_form.to_csv('prediction.csv', index=False)

# METHOD2 : DEEP LEARNING

best test accuracy = 0.42

In [5]:
train_df = pd.read_pickle("train_cleaned_df.pkl")
test_df = pd.read_pickle("test_cleaned_df.pkl")
train_df = train_df.sample(frac=1)

## Prepare data (X, y)


In [15]:
sep = int(train_df.shape[0] * 0.8) # 20% for validation

train_dataset_X = train_df[:sep]
train_dataset_y = train_df.emotion[:sep]
validation_dataset_X = train_df[sep:]
validation_dataset_y = train_df.emotion[sep:]

all_trainset_X = train_df
all_trainset_y = train_df.emotion

print(train_dataset_X.shape)
print(train_dataset_y.shape)
print(validation_dataset_X.shape)
print(validation_dataset_y.shape)

(1164450, 13)
(1164450,)
(291113, 13)
(291113,)


In [17]:
import tensorflow.keras

# standardize name (X, y) 
X_train = tfidf_vectorizer.transform(train_dataset_X['cleaned'])
y_train = train_dataset_y

X_val = tfidf_vectorizer.transform(validation_dataset_X['cleaned'])
y_val = validation_dataset_y

all_X_train =  tfidf_vectorizer.transform(all_trainset_X['cleaned'])
all_y_train = all_trainset_y

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_val.shape: ', X_val.shape)
print('y_val.shape: ', y_val.shape)

X_train.shape:  (1164450, 15000)
y_train.shape:  (1164450,)
X_val.shape:  (291113, 15000)
y_val.shape:  (291113,)


In [18]:
test_dataset_X = tfidf_vectorizer.transform(test_df['cleaned'])

In [19]:
print('x_test_data.shape: ', test_dataset_X.shape)

x_test_data.shape:  (411972, 15000)


## Deal with categorical label (y)

Rather than put your label `train_df['emotion']` directly into a model, we have to process these categorical (or say nominal) label by ourselves. 

Here, we use the basic method [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) to transform our categorical  labels to numerical ones.


In [20]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_val.shape: ', y_val.shape)



check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:7]:
 270465              joy
1413126    anticipation
1353851             joy
1587836    anticipation
389451              joy
1243511    anticipation
80836           sadness
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_val.shape:  (291113,)


In [23]:
def label_encode(le, labels):
    enc = le.transform(labels)
    return tensorflow.keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_val = label_encode(label_encoder, y_val)
all_y_train = label_encode(label_encoder, all_y_train)

print('\n\n## After convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_val.shape: ', y_val.shape)
print('all_y_train.shape: ', all_y_train.shape)



## After convert
y_train[0:7]:
 [[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (1164450, 8)
y_val.shape:  (291113, 8)
all_y_train.shape:  (1455563, 8)


## BUILD MODEL

In [24]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  15000
output_shape:  8


In [26]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 10000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=256)(X)  # 512
H1 = ReLU()(X_W1)

# 1st hidden layer
H1_W2 = Dense(units=64)(H1)  # 256
H2 = ReLU()(H1_W2)

# 2nd hidden layer
H2_W3 = Dense(units=32)(H2)  # 64
H3 = ReLU()(H2_W3)


# 2nd hidden layer
H3_W4 = Dense(units=8)(H3)  # 32
H4 = Softmax()(H3_W4)

# output layer
#H4_W5 = Dense(units=output_shape)(H4)  # 8
#H5 = Softmax()(H4_W5)

model_output = H4

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15000)]           0         
_________________________________________________________________
dense (Dense)                (None, 256)               3840256   
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 32)                0     

# TRAIN

In [27]:
from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log_DENSE.csv')

# training setting
epochs = 5
batch_size = 1000

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_val, y_val))
print('training finish')

InvalidArgumentError: indices[1] = [0,14760] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

In [ ]:
## predict
pred_result_val = model.predict(X_val, batch_size=1000)
pred_result_val[:5]

In [ ]:
pred_result_val = label_decode(label_encoder, pred_result_val)
pred_result_val[:5]

In [ ]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_val), pred_result_val), 2)))

In [ ]:
#Let's take a look at the training log
training_log = pd.DataFrame()
training_log = pd.read_csv("logs/training_log.csv")
training_log

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train accuracy', 'Val accuracy'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train loss', 'Val loss'], loc='upper left')
plt.show()

In [ ]:
pred_form = pd.read_pickle("test_df.pkl")
pred_form = pred_form.drop('hashtags', axis=1)
pred_form = pred_form.drop('text', axis=1)
pred_form = pred_form.drop('identification', axis=1)
pred_form.rename(columns={'tweet_id':'id'}, inplace=True)

In [ ]:
y_test_dense = model.predict(test_dataset_X)
y_test_dense = label_decode(label_encoder, y_test_dense)
pred_form['emotion'] = y_test_dense
pred_form.to_csv('prediction_dense.csv', index=False)